# DQD Search Routines Cookbook

This cookbook demonstrates how to run automated Double Quantum Dot (DQD) search routines using Stanza. After completing health check, DQD search is the next critical step in quantum dot tuning, automatically locating voltage regions where two quantum dots form with stable charge configurations.

## What You'll Learn

- How to configure DQD search routines in device YAML
- Understanding peak spacing computation and Coulomb blockade detection
- Running adaptive grid search to find DQD regions
- Analyzing and visualizing charge stability diagrams
- Advanced: sweeping barrier voltages for optimal DQD configurations
- Troubleshooting common issues and tuning parameters

## Overview of DQD Search Workflow

DQD search follows a systematic multi-stage workflow:

1. **Peak Spacing Computation** - Determine characteristic voltage spacing of Coulomb peaks by analyzing random sweeps
2. **Grid Generation** - Partition plunger voltage space into grid squares based on peak spacing
3. **Adaptive Grid Search** - Use machine learning classifiers to efficiently explore the grid:
   - **Stage 1:** Fast diagonal current trace to detect Coulomb blockade
   - **Stage 2:** Low-resolution charge stability diagram (CSD) if trace is promising
   - **Stage 3:** High-resolution CSD to confirm DQD and capture detailed charge transitions
4. **Intelligent Square Selection** - Prioritize regions near confirmed DQDs based on spatial clustering

Each stage builds on previous results, efficiently focusing measurement effort on promising voltage regions while avoiding exhaustive grid scanning.

## Prerequisites

Before running DQD search, you must complete device health check. DQD search requires:

- **Leakage test results**: Safe voltage bounds for sweeps
- **Global accumulation results**: Global turn-on saturation voltage for gate initialization
- **Gate characterization results**: Individual gate transition, cutoff, and saturation voltages

See the `healthcheck.ipynb` cookbook for how to complete these prerequisite routines.

## Setup

First, let's import the necessary modules:

**Note**: Stanza automatically logs all measurements and analysis results when you provide a `DataLogger` as a resource to the `RoutineRunner`. All sweep data, classifications, and DQD discoveries are saved with timestamps to the specified directory.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from stanza.routines import RoutineRunner
from stanza.routines.builtins import (
    compute_peak_spacing,
    run_dqd_search_fixed_barriers,
    run_dqd_search
)
from stanza.utils import load_device_config

## 1. Device Configuration

DQD search routines are configured in your device YAML file. Here's a complete example:

```yaml
name: "My Quantum Device"

gates:
  R1: {type: RESERVOIR, control_channel: 1, measure_channel: 1, v_lower_bound: -3.0, v_upper_bound: 3.0}
  R2: {type: RESERVOIR, control_channel: 2, measure_channel: 2, v_lower_bound: -3.0, v_upper_bound: 3.0}
  B0: {type: BARRIER, control_channel: 3, measure_channel: 3, v_lower_bound: -3.0, v_upper_bound: 3.0}
  B1: {type: BARRIER, control_channel: 4, measure_channel: 4, v_lower_bound: -3.0, v_upper_bound: 3.0}
  B2: {type: BARRIER, control_channel: 5, measure_channel: 5, v_lower_bound: -3.0, v_upper_bound: 3.0}
  P1: {type: PLUNGER, control_channel: 6, measure_channel: 6, v_lower_bound: -3.0, v_upper_bound: 3.0}
  P2: {type: PLUNGER, control_channel: 7, measure_channel: 7, v_lower_bound: -3.0, v_upper_bound: 3.0}

contacts:
  SOURCE: {type: SOURCE, control_channel: 8, measure_channel: 8, v_lower_bound: -3.0, v_upper_bound: 3.0}
  DRAIN: {type: DRAIN, control_channel: 9, measure_channel: 9, v_lower_bound: -3.0, v_upper_bound: 3.0}

routines:
  - name: device_health_check
    routines:
      # ... health check routines as in healthcheck.ipynb ...

  - name: dqd_discovery
    routines:
      - name: compute_peak_spacing
        parameters:
          gates: [P1, P2, R1, R2, B0, B1, B2]
          measure_electrode: DRAIN
          min_peak_scale: 0.05  # Minimum voltage scale to test (V)
          max_peak_scale: 0.2   # Maximum voltage scale to test (V)
          current_trace_points: 128  # Points per sweep trace
          max_number_of_samples: 30  # Max attempts per scale
          number_of_samples_for_scale_computation: 10  # Target samples
          seed: 42  # Random seed for reproducibility

      - name: run_dqd_search_fixed_barriers
        parameters:
          gates: [P1, P2, R1, R2, B0, B1, B2]
          measure_electrode: DRAIN
          current_trace_points: 128  # Points in diagonal trace
          low_res_csd_points: 16     # Low-res CSD grid size
          high_res_csd_points: 48    # High-res CSD grid size
          num_dqds_for_exit: 1       # Exit after finding 1 DQD
          include_diagonals: false   # Use 4-connected neighborhoods
          seed: 42

instruments:
  - name: qdac2
    type: GENERAL
    driver: qdac2
    ip_addr: 192.168.1.100
    slew_rate: 1.0
    nplc: 10
```

Save this to `device.yaml` in your working directory.

## 2. Running DQD Search with Fixed Barriers

The simplest workflow runs peak spacing computation followed by grid search with fixed barrier voltages.

### Step 1: Complete Health Check

First, ensure health check is complete (see `healthcheck.ipynb`):

In [ ]:
# Load device configuration
config = load_device_config("device.yaml")

# Create runner
runner = RoutineRunner(configs=[config])

# Run health check (required prerequisite)
health_results = runner.run_all(parent_routine="device_health_check")
print("Health check complete!")

### Step 2: Compute Peak Spacing

Peak spacing determines the characteristic voltage scale of Coulomb peaks, which sets the grid square size for the search:

In [ ]:
# Compute peak spacing
peak_result = runner.run("compute_peak_spacing")
peak_spacing = peak_result["peak_spacing"]

print(f"Peak spacing: {peak_spacing * 1000:.2f} mV")
print(f"Grid square size will be: {peak_spacing * 3 / np.sqrt(2) * 1000:.2f} mV")

### Step 3: Run DQD Search

Now run the adaptive grid search to find DQD regions:

In [ ]:
# Run DQD search with fixed barriers
dqd_result = runner.run("run_dqd_search_fixed_barriers")
dqd_squares = dqd_result["dqd_squares"]

print(f"Found {len(dqd_squares)} DQD region(s)!")

# Display best DQD
if dqd_squares:
    best_dqd = dqd_squares[0]  # Sorted by score
    print(f"\nBest DQD:")
    print(f"  Grid index: {best_dqd['grid_idx']}")
    print(f"  Total score: {best_dqd['total_score']:.3f}")
    print(f"  Current trace score: {best_dqd['current_trace_score']:.3f}")
    print(f"  Low-res CSD score: {best_dqd['low_res_csd_score']:.3f}")
    print(f"  High-res CSD score: {best_dqd['high_res_csd_score']:.3f}")

## 3. Understanding Peak Spacing Computation

Peak spacing computation works by:
1. Testing multiple voltage scales (from `min_peak_scale` to `max_peak_scale`)
2. For each scale, generating random diagonal sweeps through plunger voltage space
3. Classifying each trace for Coulomb blockade using ML models
4. If blockade detected, using peak detection to identify individual Coulomb peaks
5. Computing inter-peak spacings and taking the median across all successful measurements

The result determines the grid square size for the search:
```
grid_square_size = peak_spacing × 3/√2 ≈ 2.12 × peak_spacing
```

This ensures each grid square is large enough to capture multiple charge transitions.

## 4. Understanding Adaptive Grid Search

The search uses a three-stage classification hierarchy:

### Stage 1: Current Trace (Fast Screening)
- Diagonal sweep through grid square (typically 128 points)
- ML classifier: `coulomb-blockade-classifier-v3`
- Purpose: Quick elimination of unpromising regions
- If classification fails, skip to next square

### Stage 2: Low-Resolution CSD (Intermediate Check)
- 2D grid sweep (typically 16×16 = 256 points)
- ML classifier: `charge-stability-diagram-binary-classifier-v2-16x16`
- Purpose: Confirm DQD-like features with reasonable measurement cost
- If classification fails, skip to next square

### Stage 3: High-Resolution CSD (Confirmation)
- 2D grid sweep (typically 48×48 = 2,304 points)
- ML classifier: `charge-stability-diagram-binary-classifier-v1-48x48`
- Purpose: High-quality CSD for detailed characterization
- If classification succeeds, square is confirmed as DQD

This hierarchy minimizes measurement time by filtering out unpromising regions early.

## 5. Intelligent Square Selection

After the first random square, the algorithm prioritizes exploration using spatial clustering:

**Priority 1: DQD Neighbors**
- Explore unvisited neighbors of confirmed DQD squares
- Uses weighted selection based on proximity and DQD scores
- Rationale: DQDs cluster in voltage space

**Priority 2: High-Score Neighbors**
- Explore neighbors of squares with total_score ≥ 1.5
- These passed Stage 1 and possibly Stage 2 but not Stage 3
- May indicate near-miss or tunable DQD configurations

**Priority 3: Random Exploration**
- When no promising neighbors remain, randomly sample unvisited squares
- Ensures broad coverage of voltage space

The search terminates when:
- `num_dqds_for_exit` DQDs are found, OR
- `max_samples` squares are visited, OR
- All grid squares are visited

## 6. Visualizing DQD Search Results

Let's create comprehensive visualizations of the DQD search results:

In [ ]:
# Visualize the best DQD's high-resolution charge stability diagram
if dqd_squares:
    best_dqd = dqd_squares[0]
    
    # Extract high-res CSD data
    currents = np.array(best_dqd['high_res_csd_currents'])
    voltages = np.array(best_dqd['high_res_csd_voltages'])
    
    # Get voltage extent for plotting
    v_p1 = voltages[:, :, 0]  # P1 voltages
    v_p2 = voltages[:, :, 1]  # P2 voltages
    
    # Create figure
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot 1: High-res CSD
    im1 = axes[0].imshow(
        currents,
        extent=[v_p1.min(), v_p1.max(), v_p2.min(), v_p2.max()],
        origin='lower',
        aspect='auto',
        cmap='viridis'
    )
    axes[0].set_xlabel('P1 Voltage (V)')
    axes[0].set_ylabel('P2 Voltage (V)')
    axes[0].set_title(f'High-Res CSD (Score: {best_dqd["high_res_csd_score"]:.3f})')
    plt.colorbar(im1, ax=axes[0], label='Current (A)')
    
    # Plot 2: Current trace (diagonal)
    trace_currents = np.array(best_dqd['current_trace_currents'])
    trace_voltages = np.array(best_dqd['current_trace_voltages'])
    trace_distance = np.linspace(0, 1, len(trace_currents))
    
    axes[1].plot(trace_distance, trace_currents, 'b-', linewidth=2)
    axes[1].set_xlabel('Normalized Distance Along Diagonal')
    axes[1].set_ylabel('Current (A)')
    axes[1].set_title(f'Diagonal Current Trace (Score: {best_dqd["current_trace_score"]:.3f})')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nVoltage range of DQD:")
    print(f"  P1: [{v_p1.min():.4f}, {v_p1.max():.4f}] V")
    print(f"  P2: [{v_p2.min():.4f}, {v_p2.max():.4f}] V")

## 7. Advanced: Sweeping Barrier Voltages

For a comprehensive search, you can sweep barrier voltages to explore different tunnel coupling regimes. This is useful when fixed barriers don't yield DQDs or you want to find optimal barrier configurations. 

In particular, the middle barrier is swept from the transition voltage to the global turn-on saturation voltage. This is to ensure that there is sufficient tunnel coupling so that transition lines can be observed during transport measurements.

Transition lines are less prominent the closer the middle barrier is to global turn-on saturation - a regime that often leads to the formation of bias triangles. For DQD detection, these regions will be harder to detect by the ML classification model, which has been trained to pick up transition lines. Therefore, we would advise to start with higher tunnel coupling to maximise the reliablity and utility of the Coulomb Blockade classification stage of the DQD Search routine.

The `run_dqd_search` routine sweeps:
- **Outer barriers (B0, B2)**: From global turn-on saturation voltage to mean transition voltage
- **Inner barrier (B1)**: From transition voltage to global turn-on saturation voltage

At each barrier configuration, it runs full peak spacing computation and grid search.

In [ ]:
# Run DQD search with barrier voltage sweep
# WARNING: This can be time-consuming!
barrier_sweep_results = runner.run("run_dqd_search")

# Analyze results
sweep_results = barrier_sweep_results["run_dqd_search"]
print(f"Tested {len(sweep_results)} barrier configurations")

# Find configurations that yielded DQDs
successful_configs = [r for r in sweep_results if len(r['dqd_squares']) > 0]
print(f"Found DQDs in {len(successful_configs)} configuration(s)")

# Display best configuration
if successful_configs:
    best_config = max(successful_configs, key=lambda x: len(x['dqd_squares']))
    print(f"\nBest barrier configuration:")
    print(f"  Outer barriers (B0, B2): {best_config['outer_barrier_voltage']:.4f} V")
    print(f"  Inner barrier (B1): {best_config['inner_barrier_voltage']:.4f} V")
    print(f"  Peak spacing: {best_config['peak_spacing']['peak_spacing'] * 1000:.2f} mV")
    print(f"  DQDs found: {len(best_config['dqd_squares'])}")

## 8. Automatic Data Logging

When you run DQD search routines, Stanza automatically logs all measurements and classifications to disk. The data is organized in the following directory structure:

```
./data/
└── dqd_discovery/
    ├── compute_peak_spacing/
    │   ├── compute_peak_spacing.h5
    │   ├── session_metadata.json
    │   ├── measurement.jsonl          # Random sweep measurements
    │   └── analysis.jsonl             # Peak detections, spacings, summary
    └── run_dqd_search_fixed_barriers/
        ├── run_dqd_search_fixed_barriers.h5
        ├── session_metadata.json
        ├── measurement.jsonl          # Current traces, low/high-res CSDs
        └── analysis.jsonl             # ML classifications, DQD summary
```

Each routine session contains:
- **HDF5 file (.h5)**: Binary data for efficient storage and access
- **session_metadata.json**: Timestamps, device config, routine parameters
- **measurement.jsonl**: All sweep measurements with unique IDs
- **analysis.jsonl**: ML classification scores, peak detections, and summaries

Each measurement has a unique UUID that links it to its classification in the analysis log.

## 9. Parameter Tuning Guide

### Peak Spacing Parameters

**Voltage scale range (`min_peak_scale`, `max_peak_scale`)**
- Start with 50-200 mV (0.05-0.2 V)
- If no peaks found: increase range
- Peak spacing sets grid size: smaller peaks = finer grid

**Sample counts**
- `max_number_of_samples`: 30 is typical, increase for noisy devices
- `number_of_samples_for_scale_computation`: 10 balances accuracy and speed

**Current trace points**
- 128 points: good balance for most devices
- Increase to 256 for devices with fine peak structure

### Grid Search Parameters

**Resolution parameters**
- `current_trace_points`: 128 (fast screening)
- `low_res_csd_points`: 16 (256 measurements)
- `high_res_csd_points`: 48 (2,304 measurements)
- Higher resolution = better quality but slower

**Search control**
- `max_samples`: Default is 50% of grid squares
  - Increase for exhaustive search
  - Decrease for faster exploration
- `num_dqds_for_exit`: Set to 1 for quick discovery
  - Increase to find multiple DQD regions
- `include_diagonals`: 
  - `false`: 4-connected neighborhoods (default)
  - `true`: 8-connected neighborhoods (denser exploration)

## 10. Troubleshooting Guide

### Common Issues

**1. No peak spacings found**
- Check that health check completed successfully
- Verify gate characterization voltages are reasonable
- Increase `max_peak_scale` to test larger voltage ranges
- Increase `max_number_of_samples` for more attempts
- Check device connectivity and current measurements

**2. No DQDs found in grid search**
- Verify barrier voltages allow sufficient tunnel coupling
- Try `run_dqd_search` to sweep barrier voltages
- Increase `max_samples` to explore more grid squares
- Check that plunger voltage bounds are adequate
- Review low-score squares in logs - may indicate near-misses

**3. Grid search is too slow**
- Reduce resolution: lower `high_res_csd_points` (e.g., 32 instead of 48)
- Reduce `max_samples` for faster termination
- Set `num_dqds_for_exit: 1` to exit after first DQD
- Consider using `include_diagonals: false` for sparser exploration

**4. False positive DQD classifications**
- Review high-res CSD visualizations for charge stability patterns
- Consider adding manual validation of top candidates
- Check for systematic noise or measurement artifacts

**5. Voltages out of bounds during sweep**
- Check that `barrier_voltages` respect safe bounds from leakage test
- Verify gate characterization voltages are within device voltage ranges
- The algorithm automatically skips out-of-bounds grid squares

## 11. Understanding ML Classifiers

DQD search uses several specialized ML models:

### Coulomb Blockade Classifier (v3)
- **Input**: 1D current trace (typically 128 points)
- **Output**: Binary classification + confidence score
- **Purpose**: Fast screening for Coulomb blockade features
- **Use case**: Stage 1 filtering in grid search

### Coulomb Blockade Peak Detector (v2)
- **Input**: 1D current trace
- **Output**: Indices of detected Coulomb peaks
- **Purpose**: Identify individual charge transitions
- **Use case**: Peak spacing computation

### Charge Stability Diagram Classifiers
- **16×16 classifier (v2)**: Fast intermediate check with 256-point CSDs
- **48×48 classifier (v1)**: High-quality confirmation with 2,304-point CSDs
- **Output**: Binary DQD classification + confidence score
- **Purpose**: Detect characteristic DQD honeycomb pattern

All classifiers return a `score` indicating confidence. Higher scores indicate stronger feature detection.

## 12. Best Practices

1. **Always complete health check first** - DQD search requires gate characterization results
2. **Start with fixed barriers** - Use `run_dqd_search_fixed_barriers` before full barrier sweep
3. **Use appropriate resolution** - Balance measurement quality vs time:
   - Quick exploration: 16×16 or 32×32 high-res CSD
   - Publication quality: 48×48 or 64×64 high-res CSD
4. **Monitor peak spacing results** - If peak spacing seems wrong, adjust voltage scale range
5. **Review search progress** - Check logged data to understand where search is focusing
6. **Validate DQD discoveries** - Visually inspect high-res CSDs for proper DQD features
7. **Set reasonable exit conditions** - `num_dqds_for_exit=1` is often sufficient
8. **Use reproducible seeds** - Set `seed` parameter for reproducible random sweeps
9. **Save barrier configurations** - Record successful barrier voltages for future tuning
10. **Iterate on parameters** - If search fails, adjust voltage ranges or resolution

## 13. Example: Complete DQD Discovery Workflow

Here's a complete example from health check to DQD discovery:

In [ ]:
# Load configuration
config = load_device_config("device.yaml")
runner = RoutineRunner(configs=[config])

# Step 1: Health Check
print("Step 1: Running health check...")
health_results = runner.run_all(parent_routine="device_health_check")
print(f"  Leakage test passed: {health_results['leakage_test']}")
print(f"  Global turn-on saturation voltage: {health_results['global_accumulation']['global_turn_on_voltage']:.3f} V")

# Step 2: Peak Spacing
print("\nStep 2: Computing peak spacing...")
peak_result = runner.run("compute_peak_spacing")
print(f"  Peak spacing: {peak_result['peak_spacing'] * 1000:.2f} mV")

# Step 3: DQD Search
print("\nStep 3: Running DQD search...")
dqd_result = runner.run("run_dqd_search_fixed_barriers")
dqd_squares = dqd_result["dqd_squares"]
print(f"  Found {len(dqd_squares)} DQD(s)")

# Step 4: Analyze best DQD
if dqd_squares:
    print("\nStep 4: Analyzing best DQD...")
    best = dqd_squares[0]
    print(f"  Total score: {best['total_score']:.3f}")
    print(f"  Voltage range (P1): {best['high_res_csd_voltages'][0][0][0]:.4f} to {best['high_res_csd_voltages'][-1][-1][0]:.4f} V")
    print(f"  Voltage range (P2): {best['high_res_csd_voltages'][0][0][1]:.4f} to {best['high_res_csd_voltages'][-1][-1][1]:.4f} V")
    print("\nDQD discovery complete!")
else:
    print("\nNo DQDs found. Consider:")
    print("  - Adjusting barrier voltages")
    print("  - Running full barrier sweep with run_dqd_search")
    print("  - Checking device connectivity")

## Next Steps

After successful DQD discovery, you can:
- Extract operating points from high-resolution CSDs
- Begin fine-tuning of quantum dot parameters
- Perform charge sensing calibration
- Move to single-dot or double-dot physics experiments
- Explore interdot tunnel coupling by adjusting inner barrier

See other cookbooks for:
- Quantum dot fine-tuning routines
- Charge sensing and readout
- Custom routine development

## References

- [Stanza DQD Search Documentation](https://docs.conductorquantum.com/stanza/latest/core-concepts/routines/dqd-search)
- [Machine Learning for Quantum Dot Tuning](https://arxiv.org/pdf/2001.02589)
- [Autonomous Quantum Dot Tuning](https://arxiv.org/pdf/1911.10709)
- [Understanding Double Quantum Dot Features](https://arxiv.org/pdf/cond-mat/0610433)